# GPU
>

In [1]:
!nvidia-smi

Sat Jul 13 19:13:32 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.85                 Driver Version: 555.85         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8              4W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

>在PyTorch中,每个数组都有一个设备(device),我们通常将其称为环境(context),默认情况下,所有变量和相关的计算都分配给CPU,当我们跨多个服务器部署作业时,事情变得更加棘手,通过智能地将数组分配给环境,我们可以最大限度地减少在设备之间传输数据的时间,例如,当在带有GPU的服务器上训练神经网络时,我们通常希望模型的参数在GPU上

In [2]:
import torch
from torch import nn

torch.device('cpu'),torch.device('cuda'),torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [7]:
#查询可用GPU的数量
torch.cuda.device_count()

0

In [8]:
#定义两个函数,允许我们在不存在所需所有GPU的情况下运行代码
def try_gpu(i=0):
    """如果存在返回gpu(i),否则返回cpu()"""
    if torch.cuda.device_count()>=i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')
def try_all_gpus():
    """返回所有可用的GPU,如果没有GPU返回[cpu(),]"""
    devices=[torch.device(f'cuda{i}')for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(),try_gpu(10),try_all_gpus()

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

In [9]:
#默认情况下,张量是在cpu上创建的
x=torch.tensor([1,2,3])
x.device

device(type='cpu')

>需要注意的是,无论何时我们要对多个项进行操作,它们都必须在同一个设备上,例如我们对两个张量求和,我们需要确保两个张量在同一个设备上,否则框架将不知道在哪里存储结果,甚至不知道在哪里计算
### 存储在GPU上
>有几种方法可以在GPU上存储张量,例如我们可以在创建张量时指定存储设备

In [10]:
#在第一个GPU上创建张量变量x,在GPU上的张量只消耗这个GPU的显存
X=torch.ones(2,3,device=try_gpu())
X
#为什么没有GP

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [11]:
import torch

# 检查 GPU 是否可用
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("GPU is available.")
else:
    device = torch.device('cpu')
    print("GPU is not available, using CPU.")

# 创建一个张量并将其移动到 GPU
x = torch.randn(3, 3).to(device)
print("Tensor on device:", x)


GPU is not available, using CPU.
Tensor on device: tensor([[-1.1528,  1.2987, -0.9903],
        [-0.1720,  1.4885,  1.5777],
        [-0.9834, -1.1857, -0.1701]])


>虽然有GPU但是不能够进行使用

In [12]:
print(torch.cuda.is_available()) #检查cuda是否可用

False


In [14]:
!nvidia-smi

Sat Jul 13 21:42:16 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.85                 Driver Version: 555.85         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8              5W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----